## Relative rotation estimation and Rotation averaging 

In [1]:
import numpy as np
import cv2

In [1]:
#importing the opencv module  
import cv2  

path = '../data/img/dog.jpg'
# using imread('path') and 1 denotes read as  color image  
img = cv2.imread(path,1)  
#This is using for display the image  
cv2.imshow('image',img)  
cv2.waitKey() # This is necessary to be required so that the image doesn't close immediately.  
#It will run continuously until the key press.  
cv2.destroyAllWindows() 

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/ignaciozm/.local/lib/python3.10/site-packages/cv2/qt/plugins"


In [5]:
import cv2
import numpy as np

#importing the opencv module  
import cv2  
# using imread('path') and 1 denotes read as  color image  
img = cv2.imread(path,0)  
# get image height, width
(h, w) = img.shape[:2]
# calculate the center of the image
center = (w / 2, h / 2)
 
scale = 1.0
 
# Perform the counter clockwise rotation holding at the center
# 45 degrees
M = cv2.getRotationMatrix2D(center, 45, scale)
print(M)
rotated45 = cv2.warpAffine(img, M, (h, w))
 
# 110 degrees
M = cv2.getRotationMatrix2D(center,180, scale)
rotated180 = cv2.warpAffine(img, M, (w, h))
 
# 150 degrees
M = cv2.getRotationMatrix2D(center, 150, scale)
rotated150 = cv2.warpAffine(img, M, (h, w))
 
 
cv2.imshow('Original Image',img)
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destroys the window showing image
 
cv2.imshow('Image rotated by 45 degrees',rotated45)
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destroys the window showing image
 
cv2.imshow('Image rotated by 110 degrees',rotated180)
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destroys the window showing image
 
cv2.imshow('Image rotated by 150 degrees',rotated150)
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destroys the window showing image

[[  0.70710678   0.70710678 -24.42745289]
 [ -0.70710678   0.70710678 124.02691193]]


In [6]:
# Numpy/OpenCV implementation
import cv2
from scipy import signal
import scipy.fft as sfft
import numpy as np


# OpenCV implementation
def find_rotation_cv(img_ref: np.ndarray, img_moved: np.ndarray) -> float:
    h, w = img_ref.shape[:2]
    hamming_w = signal.windows.hamming(w)
    hamming_h = signal.windows.hamming(h)
    hamming = np.outer(hamming_h, hamming_w)

    F_ref = np.log(np.abs(sfft.fftshift(sfft.fft2(img_ref * hamming))))
    F_moved = np.log(np.abs(sfft.fftshift(sfft.fft2(img_moved * hamming))))

    center_x = w // 2
    center_y = h // 2
    radius = min(center_x, center_y)

    # Define the desired size of the output polar image
    polar_width = radius
    polar_height = int(np.ceil(radius * np.pi / 2))

    # Perform the polar transformation
    F_ref_warpped = cv2.warpPolar(
        F_ref, 
        (polar_width, polar_height), 
        (center_x, center_y), 
        radius, 
        cv2.WARP_POLAR_LOG + cv2.INTER_CUBIC + cv2.WARP_FILL_OUTLIERS
    )
    F_moved_warpped = cv2.warpPolar(
        F_moved, 
        (polar_width, polar_height), 
        (center_x, center_y), 
        radius, 
        cv2.WARP_POLAR_LOG + cv2.INTER_CUBIC + cv2.WARP_FILL_OUTLIERS
    )

    ret = cv2.phaseCorrelate(F_ref_warpped[:180], F_moved_warpped[:180])
    theta_shift = 360 / polar_height * -ret[0][1]
    return theta_shift


In [11]:
print('shape of rotated180', rotated180.shape, 'shape of img', img.shape), 
find_rotation_cv(img, rotated180)

shape of rotated180 (183, 275) shape of img (183, 275)


0.04617900679547277

In [15]:
def predict_rotation(img1, img2):
    # Predicts the rotation between two images
    # img1: first image
    # img2: second image
    # returns: predicted rotation in degrees

    # Convert images to grayscale
    if len(img1.shape) > 2:
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2, None)

    # Match features.
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(descriptors1, descriptors2, None)

    # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)

    # Remove not so good matches
    numGoodMatches = int(len(matches) * 0.15)
    matches = matches[:numGoodMatches]

    # Draw top matches
    imgMatches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches, None)
    cv2.imshow("Matches", imgMatches)
    cv2.waitKey()

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Use homography
    height, width, channels = img2.shape
    img1Reg = cv2.warpPerspective(img1, h, (width, height))

    return img1Reg, h

predict_rotation(rotated180, rotated180)

AttributeError: 'tuple' object has no attribute 'sort'

In [27]:

class EstimateRot:
    def __init__(self):
        self.K = np.array([[246.43059488719612, 0, 103.08424033768227],
                           [0, 245.6414223852361, 77.55187058195787],
                           [0, 0, 1]], dtype=np.float32)

    def predict_rotation(self, predicted, real):
        p2 = np.array(predicted)
        p1 = np.array(real)
        p1dbl = np.hstack((p1, np.ones((len(p1), 1))))
        p2dbl = np.hstack((p2, np.ones((len(p2), 1))))
        prediction = np.float32(p2dbl)
        reality = np.float32(p1dbl)
        A = np.zeros((3, 3), dtype=np.float32)
        tform = np.eye(3, dtype=np.float32)
        reality = cv2.gemm(cv2.invert(self.K)[1], reality.T, 1, tform, 0)
        prediction = cv2.gemm(cv2.invert(self.K)[1], prediction.T, 1, tform, 0)
        A = cv2.gemm(reality, prediction.T, 1, tform, 0)
        w, u, vt = cv2.SVDecomp(A)
        u, v = vt.T, u.T
        A = cv2.gemm(v, u, 1, tform, 0)
        det = np.linalg.det(A)
        if det < 0:
            v[:, 2] *= -1
            A = cv2.gemm(v, u, 1, tform, 0)
            det = np.linalg.det(A)
        tform[2, 2] = det
        temp2 = A
        temp2 = cv2.gemm(v, tform, 1, tform, 0)
        tform = cv2.gemm(temp2, u, 1, tform, 0)
        return tform
    
    def get_rotation(self, predicted, real):
        tform = self.predict_rotation(predicted, real)
        w, u, vt = cv2.SVDecomp(tform)
        u, v = vt.T, u.T
        R = cv2.gemm(v, u, 1, tform, 0)
        return R
    
    def get_rotation_angle(self, predicted, real):
        R = self.get_rotation(predicted, real)
        angle = np.arccos((np.trace(R) - 1) / 2)
        return angle


EstimateRot().get_rotation_angle(rotated180, rotated180)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)